# [Pytorch Official tutorial](https://docs.pytorch.org/tutorials/beginner/basics/intro.html)

### Working with data

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
#Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root= "data",
    train = True,
    download = True,
    transform = ToTensor() 
)

#Download test data from  open datasets.
test_data = datasets.FashionMNIST(
    root = "data",
    train = False,
    download = True,
    transform = ToTensor()
)

In [3]:
batch_size = 64

#create data loaders.
train_dataloader = DataLoader(training_data,batch_size = batch_size)
test_dataloader = DataLoader(test_data,batch_size=batch_size)

for X,y in test_dataloader:
    print(f"Shape of X [N.C,H,W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N.C,H,W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


### Creating Model

In [4]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

#Define Model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28,16),
            nn.ReLU(),
            nn.Linear(16,16),
            nn.ReLU(),
            nn.Linear(16,10)
        )

    def forward(self,x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=16, bias=True)
    (1): ReLU()
    (2): Linear(in_features=16, out_features=16, bias=True)
    (3): ReLU()
    (4): Linear(in_features=16, out_features=10, bias=True)
  )
)


### Optimizing the model parameters.
To train a model, we need a loss function and an optimizer.

In [5]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr=1e-3)

In a single training loop, the model makes predictions on the training datasets(fed to it in batches), and backpropagates the prediction error to adjust the model's parameter.

In [6]:
def train(dataloader,model,loss_fn,optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch,(X,y) in enumerate(dataloader):
        X,y = X.to(device),y.to(device)

        #Compute prediction error
        pred = model(X)
        loss = loss_fn(pred,y)

        #Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [7]:
def test(dataloader,model,loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss,correct = 0,0
    with torch.no_grad():
        for X,y in dataloader:
            X,y = X.to(device),y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred,y).item()
            correct += (pred.argmax(1)==y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}, Avg loss: {test_loss:>8f} \n")

The training process is conducted over several iterations(epochs). During each epoch, the model learn parameters to make better predictions. We print the model's accuracy and loss at each epoch; we'd like to see the accuracy increase and the loss decrease with every epoch.

In [8]:
epochs = 30
for t in range(epochs):
    print(f"Epoch {t+1}\n----------------------------")
    train(train_dataloader,model,loss_fn,optimizer)
    test(test_dataloader,model,loss_fn)
print("Done!")

Epoch 1
----------------------------
loss: 2.290932 [   64/60000]
loss: 2.282536 [ 6464/60000]
loss: 2.287373 [12864/60000]
loss: 2.292850 [19264/60000]
loss: 2.295184 [25664/60000]
loss: 2.297304 [32064/60000]
loss: 2.278300 [38464/60000]
loss: 2.283965 [44864/60000]
loss: 2.253736 [51264/60000]
loss: 2.294229 [57664/60000]
Test Error: 
 Accuracy: 19.1, Avg loss: 2.263598 

Epoch 2
----------------------------
loss: 2.234006 [   64/60000]
loss: 2.227230 [ 6464/60000]
loss: 2.231144 [12864/60000]
loss: 2.251930 [19264/60000]
loss: 2.244124 [25664/60000]
loss: 2.244475 [32064/60000]
loss: 2.227018 [38464/60000]
loss: 2.223637 [44864/60000]
loss: 2.182250 [51264/60000]
loss: 2.227655 [57664/60000]
Test Error: 
 Accuracy: 22.0, Avg loss: 2.195410 

Epoch 3
----------------------------
loss: 2.152727 [   64/60000]
loss: 2.147679 [ 6464/60000]
loss: 2.147754 [12864/60000]
loss: 2.184221 [19264/60000]
loss: 2.153718 [25664/60000]
loss: 2.143822 [32064/60000]
loss: 2.131433 [38464/60000]
loss

### Saving Models
A common way to save a model is to serialize the internal state dictionary(containing the model parametrs).

In [9]:
torch.save(model.state_dict(),"model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


### Loading Models
The process for loading a model includes re-creating the model structure and loading the state dictionary into it.

In [10]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth",weights_only=True))

<All keys matched successfully>

This model can now be used to make predictions.

In [11]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot"
]

model.eval()
x,y = test_data[0][0],test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted,actual = classes[pred[0].argmax(0)],classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
